# Reference used: 
- https://dzone.com/articles/algorithm-week-homomorphic

## Sub References: 
- https://www.cs.cmu.edu/~rwh/theses/okasaki.pdf
- https://pdos.csail.mit.edu/papers/otfvec/paper.pdf, 


## Properties of a Homomorphic Function: 

    k.H(m + n + o) = k.H(m) + k.H(n) + k.H(o) = H(k.m) + H(k.n) + H(k.o)

    The homomorphic hash function is:
        H(x) = g^x (mod q)

    here, sum = a + b + c + d + e -> sum is the key and
    a, b, ..., e are the subkeys that are generated from the key

In [71]:
import random
import math
import pickle

PASS_LEN = 10
BLOCK_LEN = 5

In [72]:
def isPrime(x: int) -> bool:
    '''
    Checks if the given number is a prime
    '''
    for i in range(2, int(math.sqrt(x)) + 1):
        if(x % i == 0):
            return False

    return True

In [73]:
def primegen(start = 1001, step = 1) -> list:
    '''
    Generates a set of primes P to be used for for hashing
    '''
    i = start
    primes = []
    while (len(primes) < BLOCK_LEN):
        if(isPrime(i)):
            primes.append(i)
        i+=step
    # print(f'primes({start}, {step}): {primes}')

    return primes

In [74]:
def keygen(P = 1001) -> int:
    '''
    Generates a set of primes Q to be used for hashing
    Q follows the property: Q % P == 1
    '''
    keys = []
    i = 1
    # print(f"P: {P}\nQ:")
    while len(keys) < BLOCK_LEN:
        Q = P * i + 1
        if(isPrime(Q) and Q % P == 1):
            # print(f"\t{Q}")
            keys.append(Q)
        i+=1
    k = random.randint(0, BLOCK_LEN-1)
    # print(f'keygen({P}): {keys} - {keys[k]}')
    
    return keys[k]

In [75]:
def HomoHash(a: int, b: int, x: int, k = 1) -> int:
    '''
    The main homomorphic hash function: a^(k.x) (mod b)
    '''
    return pow(a, k*x, b)

In [76]:
def obfuscate(x: list, k = [1 for j in range(BLOCK_LEN)]) -> list:
    '''
    Obfuscates all the subkeys to be sent by multiplying them with a random secret number k
    '''
    obfuscated_message = [[0 for j in range(BLOCK_LEN)] for i in range(len(x))] 
    for i in range(len(x)):
        for j in range(BLOCK_LEN):
            obfuscated_message[i][j] = x[i][j]*k[j]
            # print(f"obfuscated_message[{i}][{j}] = x[{i}][{j}]*k[{j}] = {x[i][j]}*{k[j]} = {obfuscated_message[i][j]}")

    return obfuscated_message

In [77]:
def client_hash(x: list, g: list, q: list, p: list, k = [1 for j in range(BLOCK_LEN)])  -> list:
    '''
    implements "add": g^k(a+b) (mod q)
    # to compute the hash:
    ## compute the sum of the characters by adding each one
    ## hash that sum by raising g to the power of the sum (mod q)
    '''
    sums = [0 for j in range(BLOCK_LEN)]
    hashed_sums = [0 for j in range(BLOCK_LEN)]
    for i in range(BLOCK_LEN): # for i in range(0, PASS_LEN//BLOCK_LEN + 1): sum+= i%p
        # sum = 0 and replace hashed_sums with sum
        for j in range(len(x)):
            sums[i] += (x[j][i] % p[i])
        sums[i] %= p[i]
        hashed_sums[i] = HomoHash(g[i], q[i], k[i]*(sums[i]))
    # print(hashed_sums)
    
    return hashed_sums

In [78]:
def server_hash(x: list, g: list, q: list, p: list, k = [1 for j in range(BLOCK_LEN)])  -> tuple:
    '''
    implements "multiply": g^k.a * g^k.b (mod q)
    # hashed_products are the keys computed by each individual server
    # to compute the hash:
    ## compute the hash of each individual character by raising g to the power of the character (mod q)
    ## compute the "sum" of the hashes of the individual characters by multiplying them (mod q)
    '''
    hashed_products = [[1 for j in range(BLOCK_LEN)] for i in range(len(x))]
    products = [1 for j in range(BLOCK_LEN)]
    # for i in range(0, PASS_LEN//BLOCK_LEN + 1): product *= pow(g, q, x[i]%p)
    for i in range(BLOCK_LEN):
        for j in range(len(x)):
            temp = HomoHash(g[i], q[i], k[i]*x[j][i])
            hashed_products[j][i] = temp
            products[i] *= temp
            products[i] %= q[i]
    # print(products)
    # print(hashed_products)

    return products, hashed_products

In [79]:
# Variables used in the script
# get password from the user
inp = input("10 Characters: ") # private
# 'qwertyuiop'  
inp += ' '*(PASS_LEN-len(inp))
# hash message
m = [[ord(inp[i+j]) for j in range(BLOCK_LEN)] for i in range(0, PASS_LEN, BLOCK_LEN)] # private
S = [[0 for j in range(BLOCK_LEN)] for i in range(len(m))] # distributed

10 Characters: qwertyuiop


In [80]:
# Constants used in the hash function
## Choose prime p - 257
p = primegen(1001) # public

In [81]:
## Choose q such that `q % p == 1` or `p | (q - 1)` - 257*6 + 1
q = [keygen(x) for x in p] # public

keygen(1009): [10091, 12109, 30271, 40361, 42379] - 12109
keygen(1013): [2027, 6079, 12157, 20261, 26339] - 6079
keygen(1019): [2039, 32609, 38723, 50951, 61141] - 61141
keygen(1021): [10211, 12253, 18379, 30631, 40841] - 40841
keygen(1031): [2063, 12373, 30931, 32993, 37117] - 37117


In [82]:
## a random number g
g = [random.randint(50, 100) for i in range(BLOCK_LEN)] # public

In [ ]:
## generate fuzz factors k
k = [random.randint(15, 50) for i in range(BLOCK_LEN)] # private

In [83]:
# Calculate all the hashes from all parties
client_tmp = client_hash(m, g, q, p)
client_tmp_obf = client_hash(m, g, q, p, k)
server_tmp = server_hash(m, g, q, p)
server_tmp_obf = server_hash(m, g, q, p, k)

In [88]:
print("Data generated: ")
print("\n\tInput: ")
print("\t",inp)
print("\n\tInput encoded into blocks: ")
print("\t",m)
print("\n\tPrimes generated (P): ")
print("\t",p)
print("\n\tDependent primes (Q: Q % P == 1): ")
print("\t",g)
print("\n\tFuzz factors: ")
print("\t",k)
print("\n")

print("Clean data: ")
print("\n\tClient side:")
print("\t",client_tmp)
print("\n\tServer side: ")
print("\n\tIndividual server computation: ")
print("\t",server_tmp[1])
print("\n\tNetwork wide computation: ")
print("\t",server_tmp[0])
if(client_tmp == server_tmp[0]):
    print("\n\tHashes are equal!")
print("\n")

print("Obfuscated data: ")
print("\n\tClient side:")
print("\t",client_tmp_obf)
print("\n\tServer side: ")
print("\n\tIndividual server computation: ")
print("\t",server_tmp_obf[1])
print("\n\tNetwork wide computation: ")
print("\t",server_tmp_obf[0])
if(client_tmp_obf == server_tmp_obf[0]):
    print("\n\tObfuscated hashes are equal!")
print("\n")

Data generated: 

	Input: 
	 qwertyuiop

	Input encoded into blocks: 
	 [[113, 119, 101, 114, 116], [121, 117, 105, 111, 112]]

	Primes generated (P): 
	 [1009, 1013, 1019, 1021, 1031]

	Dependent primes (Q: Q % P == 1): 
	 [58, 73, 80, 78, 57]

	Fuzz factors: 
	 [30, 21, 33, 32, 40]


Clean data: 

	Client side:
	 [10552, 2062, 39610, 34827, 4925]

	Server side: 

	Individual server computation: 
	 [[6853, 2106, 9262, 20574, 25675], [6117, 2137, 52458, 21113, 25805]]

	Network wide computation: 
	 [10552, 2062, 39610, 34827, 4925]

	Hashes are equal!


Obfuscated data: 

	Client side:
	 [485, 1043, 23172, 30269, 28928]

	Server side: 

	Individual server computation: 
	 [[7828, 3030, 5660, 6260, 374], [9187, 1170, 35306, 19388, 5238]]

	Network wide computation: 
	 [485, 1043, 23172, 30269, 28928]

	Obfuscated hashes are equal!




In [89]:
'''
# Reference used: 
## https://dzone.com/articles/algorithm-week-homomorphic
## Sub References: https://www.cs.cmu.edu/~rwh/theses/okasaki.pdf, https://pdos.csail.mit.edu/papers/otfvec/paper.pdf, 

# Properties of a Homomorphic Function: kH(m + n + o) = H(km) + H(kn) + H(ko) = kH(m) + kH(n) + kH(o)
# the homomorphic hash function is H(x) = g^x mod q
# here, sum = a + b + c + d + e -> sum is the key and a, b, ..., e are the subkeys that are generated from the key

# p  [1 2 3 4 5]
# q  [1 2 3 4 5]
# g  [1 2 3 4 5]
# h  [1 2 3 4 5]
# m [[1 2 3 4 5]
#    [6 7 8 9 0]]
# Hs [1 2 3 4 5]
# S [[1 2 3 4 5]
#    [6 7 8 9 0]]
# Hp [1 2 3 4 5]

# test = [[113, 119, 101, 114, 116], [121, 117, 105, 111, 112]]
# print(obfuscate(test))

# Hp = [0 for j in range(BLOCK_LEN)] # public
# Hs = [0 for j in range(BLOCK_LEN)] # public

Hs1 = client_hash(m, g, q, p)

# hashes computed at each server, same length as the input + padding
Hp1 = server_hash(m, g, q, p)

Hs = client_hash(m, g, q, p, k)

# hashes computed at each server, same length as the input + padding - it has also been obfuscated

Hp = server_hash(m, g, q, p, k)

print(f"""\nmessage (primary key):\n{obfuscate(m)}\n
obfuscated sub keys (sent to each server):\n{obfuscate(m, k)}\n
hashes (generated by each server):\n{S}\n
sum_hash (hashing entire message - done by client):\n{Hs1}\n
product (hashing each block and then adding the hashes - done by the distributed network):\n{Hp1}\n
k_sum (obfuscated sum_hash):\n{Hs}\n
k_product (obfuscated product_hash):\n{Hp}\n
k_server (obfuscated hashes calculated on the server):\n{S1}""")

if(sum_hash == product_hash):
    print("Hashes are equal!")

if(k_sum == k_product):
    print("Obfuscated hashes are equal!")

'''

'\n# Reference used: \n## https://dzone.com/articles/algorithm-week-homomorphic\n## Sub References: https://www.cs.cmu.edu/~rwh/theses/okasaki.pdf, https://pdos.csail.mit.edu/papers/otfvec/paper.pdf, \n\n# Properties of a Homomorphic Function: kH(m + n + o) = H(km) + H(kn) + H(ko) = kH(m) + kH(n) + kH(o)\n# the homomorphic hash function is H(x) = g^x mod q\n# here, sum = a + b + c + d + e -> sum is the key and a, b, ..., e are the subkeys that are generated from the key\n\n# p  [1 2 3 4 5]\n# q  [1 2 3 4 5]\n# g  [1 2 3 4 5]\n# h  [1 2 3 4 5]\n# m [[1 2 3 4 5]\n#    [6 7 8 9 0]]\n# Hs [1 2 3 4 5]\n# S [[1 2 3 4 5]\n#    [6 7 8 9 0]]\n# Hp [1 2 3 4 5]\n\n# test = [[113, 119, 101, 114, 116], [121, 117, 105, 111, 112]]\n# print(obfuscate(test))\n\n# Hp = [0 for j in range(BLOCK_LEN)] # public\n# Hs = [0 for j in range(BLOCK_LEN)] # public\n\nHs1 = client_hash(m, g, q, p)\n\n# hashes computed at each server, same length as the input + padding\nHp1 = server_hash(m, g, q, p)\n\nHs = client_h